In [5]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tranh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tranh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tranh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tranh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tranh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
df = pd.read_csv('../../data_use/bbc_text_cls.csv')

print(f"Số lượng văn bản ban đầu: {len(df)}")
print(f"\nCột trong dataset: {df.columns.tolist()}")
print(f"\nCác nhãn: {df['labels'].unique()}")
print(f"\nPhân bố nhãn:\n{df['labels'].value_counts()}")

Số lượng văn bản ban đầu: 2225

Cột trong dataset: ['text', 'labels']

Các nhãn: ['business' 'entertainment' 'politics' 'sport' 'tech']

Phân bố nhãn:
labels
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64


In [7]:
# Khởi tạo các công cụ xử lý
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    """
    Hàm làm sạch văn bản
    """
    # Chuyển về chữ thường
    text = text.lower()
    
    # Loại bỏ ký tự đặc biệt và dấu câu (chỉ giữ lại chữ cái và khoảng trắng)
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Loại bỏ stopwords và lemmatization
    cleaned_tokens = []
    for token in tokens:
        if token not in stop_words and len(token) > 1:  # Loại bỏ từ có 1 ký tự
            lemmatized = lemmatizer.lemmatize(token)
            cleaned_tokens.append(lemmatized)
    
    # Ghép lại thành văn bản
    cleaned_text = ' '.join(cleaned_tokens)
    
    return cleaned_text

In [8]:
# Áp dụng làm sạch dữ liệu
print("\n" + "="*50)
print("Đang làm sạch dữ liệu...")
print("="*50)

df['cleaned_text'] = df['text'].apply(clean_text)

# Tính độ dài văn bản (số từ)
df['word_count'] = df['cleaned_text'].apply(lambda x: len(x.split()))

print(f"\nThống kê độ dài văn bản (số từ):")
print(df['word_count'].describe())

# Lọc văn bản có độ dài <= 1000 từ
outliers_count = len(df[df['word_count'] > 1000])
print(f"\nSố văn bản có độ dài > 1000 từ (outliers): {outliers_count}")

df_cleaned = df[df['word_count'] <= 1000].copy()
print(f"Số văn bản sau khi loại bỏ outliers: {len(df_cleaned)}")


Đang làm sạch dữ liệu...

Thống kê độ dài văn bản (số từ):
count    2225.000000
mean      216.142022
std       125.279657
min        48.000000
25%       140.000000
50%       189.000000
75%       267.000000
max      2211.000000
Name: word_count, dtype: float64

Số văn bản có độ dài > 1000 từ (outliers): 7
Số văn bản sau khi loại bỏ outliers: 2218

Thống kê độ dài văn bản (số từ):
count    2225.000000
mean      216.142022
std       125.279657
min        48.000000
25%       140.000000
50%       189.000000
75%       267.000000
max      2211.000000
Name: word_count, dtype: float64

Số văn bản có độ dài > 1000 từ (outliers): 7
Số văn bản sau khi loại bỏ outliers: 2218


In [9]:
# Kiểm tra kết quả
print("\n" + "="*50)
print("Ví dụ văn bản gốc và văn bản đã làm sạch:")
print("="*50)
idx = 0
print(f"\nNhãn: {df_cleaned.iloc[idx]['labels']}")
print(f"\nGốc:\n{df_cleaned.iloc[idx]['text'][:300]}...")
print(f"\nĐã làm sạch:\n{df_cleaned.iloc[idx]['cleaned_text'][:300]}...")
print(f"\nSố từ: {df_cleaned.iloc[idx]['word_count']}")

# Lưu dữ liệu đã làm sạch
df_cleaned[['cleaned_text', 'labels']].to_csv('../../data_use/bbc_cleaned.csv', index=False)
print("\n" + "="*50)
print("Đã lưu dữ liệu đã làm sạch vào file: bbc_cleaned.csv")
print("="*50)

# Thống kê cuối cùng
print(f"\nPhân bố nhãn sau khi làm sạch:")
print(df_cleaned['labels'].value_counts())
print(f"\nTỷ lệ giữ lại: {len(df_cleaned)/len(df)*100:.2f}%")


Ví dụ văn bản gốc và văn bản đã làm sạch:

Nhãn: business

Gốc:
Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and highe...

Đã làm sạch:
ad sale boost time warner profit quarterly profit u medium giant timewarner jumped bn three month december yearearlier firm one biggest investor google benefited sale highspeed internet connection higher advert sale timewarner said fourth quarter sale rose bn bn profit buoyed oneoff gain offset prof...

Số từ: 239

Đã lưu dữ liệu đã làm sạch vào file: bbc_cleaned.csv

Phân bố nhãn sau khi làm sạch:
labels
sport            511
business         510
politics         414
tech             400
entertainment    383
Name: count, dtype: int64

Tỷ lệ giữ lại: 99.69%
